In [78]:
# Initialize Otter
import otter
grader = otter.Notebook()

# DATA 601: HW3

## Fall 2020

### Due: Wed. Oct. 7, 2020 (by 23:55) 

**Learning Objectives**
- Work with realworld datasets that can be represented using tabular data structures.
- Gain experience wrangling and organizing data using `pandas`.
- Produce visualizations summarizing information from tabular data. 

_This is an individual homework assignment._ 

Please complete this homework assignment within the Jupypter notebook environment, and provide your answers where indicated.

#### Submission 

Your submission will be manually tested and graded. In order to ensure that everything goes smoothly, please ensure that: 

- your notebook runs on the Data Science Hub environment, 
- you have provided your solutions where asked without altering any other parts of this notebook,
- you have followed the instructions outlined in the last cell of this notebook to prepare your work for submission on Gradescope.ca.

_You do not need to submit any datasets._

## Warm up
 
- Please review the Calgary Rainfall Jupyter notebook. In this homework, we will
  use the Calgary Rainfall dataset. Please download the dataset if you already haven't done so. You can download the dataset from [Open Calgary](https://data.calgary.ca/Environment/Historical-Rainfall/d9kv-swk3). 

<!-- BEGIN QUESTION -->

## Imports

In order to assist the grader with testing your solution, please use the cell below to import any packages needed by your notebook. 

<!--
BEGIN QUESTION
name: t0_imports
manual: true
-->

In [79]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot')
import calendar

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Task 1 (5 points)

### Cleanup and organization

- Use `pandas` to read in the data set. Do not discard the datetime information in the columns. Convert the 'TIMESTAMP' column to a `datetime` object (You can use [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) to accomplish this).

- You may notice that 'YEAR' column is now redundant. Additionally, for this homework, we won't make use of the 'NAME', 'ID' and 'DATA TYPE' columns. Please discard columns that are not needed to save memory (you can use [`pandas.DataFrame.drop`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html) for this). 

- Discard any rows where the channel is not active. Now the 'RG_ACTIVE' column is no longer needed, so discard that as well.

- Display the head ([`pandas.DataFrame.head`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html)), tail ([`pandas.DataFrame.tail`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.tail.html)) and description ([`pandas.DataFrame.describe`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)) of the resulting dataframe.

Please provide your solution by inserting appropriate code in the cell below:

<!--
BEGIN QUESTION
name: t1_cleanup
manual: true
-->

In [ ]:
rdata = pd.read_csv("Historical_Rainfall.csv")
rdata['TIMESTAMP'] = pd.to_datetime(rdata['TIMESTAMP'])
A = rdata.drop(['YEAR', 'NAME','DATA TYPE','ID'],axis = 1)

A = A.drop(A.loc[A['RG_ACTIVE']!='Y'].index)

A = A.drop(['RG_ACTIVE'],axis = 1)
display(A.head())

display(A.tail())
display(A.describe())

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Task 2 (10 points)

### Restructure and determine rainfall daily totals per channel

- We are interested in the daily rainfall totals per channel. Restructure and aggregate your table so that entries now contain _daily totals per channel_.  

The precise details of how you accomplish this are up to you. You can for example build a hierarchical index for the rows with the year, month and day. You can also have a hierarchical index on the columns based on the channels. Please make use of `pandas` grouping and aggregation facilities to accomplish this.

You may find the following useful:
- [pandas.Series.dt.year](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.year.html)
- [pandas.Series.dt.month](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.month.html)
- [pandas.Series.dt.day](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.day.html)



Please provide your solution by inserting appropriate code in the cell below.

<!--
BEGIN QUESTION
name: t2_restructure
manual: true
-->

In [ ]:
date = pd.Series(A['TIMESTAMP'])
date = pd.to_datetime(date)

Y = date.dt.year
M = date.dt.month
D = date.dt.day

adata = pd.DataFrame({'CHANNEL' : A['CHANNEL'], 'YEAR' : Y, 'MONTH' : M, 'DAY':D, 'RAINFALL' : A['RAINFALL']})

grouped = adata.groupby(['YEAR', 'MONTH','DAY','CHANNEL'])
totalDL0 = grouped.agg({"RAINFALL":"sum"})
totalDL = grouped.sum().reset_index()
display(totalDL0.head())
display(totalDL0.tail())


<!-- END QUESTION -->



## Task 3 (15 points)

### Visualization

Produce visualizations that show:

A. Rainiest day of the year for the years 1989 through 2019:  
For each day, show the date and the total rainfall. Note that you will need to aggregate over the
channels. Use the mean over the channels for this, i.e. _total rainfall for a particular day is
taken to be the average of the total rainfall recorded by each channel that has recorded rainfall
for that day_.  

B. Average number of rainy days per month:  
Average over the years 1989 through 2019. _Take a rainy day to be one for which the total rainfall recorded (as defined above) is 2.5mm or more_.  

C. Rainfall *monthly* statistics such as mean, median, min and max:  
In order words, for a particular   month, what is the mean rainfall, the median rainfall, the minimum rainfall and the maximum rainfall. Again, use the data for the years 1989 through 2019. You will need to aggregate over the channels as well - again use the mean.

The details of what visualizations to use are not spelled out. Please choose a visualization that is appropriate for each of the above tasks and *clearly* shows the requested information. Please also ensure that you provide appropriate labels/legends/colorbars so that your visualizations are readable and self-contained.

<!-- BEGIN QUESTION -->

Please provide your solution to Task 3A in the code cell below:

<!--
BEGIN QUESTION
name: t3_A
manual: true
-->

In [ ]:
totalDL = totalDL[(totalDL.YEAR>1988)&(totalDL.YEAR<2020)]
ave_day_rf = totalDL.groupby(['YEAR', 'MONTH','DAY']).agg({'RAINFALL':'mean'})
rainest_day_of_year = ave_day_rf.loc[ave_day_rf.groupby('YEAR')['RAINFALL'].idxmax()]
fig,ax1 = plt.subplots()
rainest_day_of_year.plot(kind='bar', color='b', grid=True,ax=ax1)
fig.set_size_inches(15,6)
plt.xlabel("Date")
plt.ylabel("RAINFALL")
plt.title("Rainest Day of a Year")
plt.legend()
plt.show()

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Please provide your solution to Task 3B in the code cell below:

<!--
BEGIN QUESTION
name: t3_B
manual: true
-->

In [ ]:
rainyday = ave_day_rf.loc[ave_day_rf['RAINFALL'] >2.5]

rainy = rainyday.groupby(level=[1,0]).size()
rainy = rainy.groupby(['MONTH']).mean()
rainy= rainy.reset_index()
rainy = pd.DataFrame(rainy)
rainy.columns =['MONTH', 'DAYS'] 
rainy['MONTH'] = rainy['MONTH'].apply(lambda t: calendar.month_abbr[t])
display(rainy)
fig,ax1 = plt.subplots()
rainy['DAYS'].plot(kind='bar', color='b', grid=True,ax=ax1)
fig.set_size_inches(15,6)
plt.xticks(rainy.index,rainy['MONTH'].values)
plt.ylabel("Average Days")
plt.title('Average Rainy Day per Month')
plt.show()


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Please provide your solution to Task 3C in the code cell below:

<!--
BEGIN QUESTION
name: t3_C
manual: true
-->

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
Data =  ave_day_rf.groupby(level=[0,1]).sum()
ax.set_title('Month Rainfall Statistics')
Data = Data.unstack()
Data.boxplot( rot=360, fontsize=12, showmeans=True,labels = ['May','June','July','August','September'])
plt.xticks(np.arange(1, 6, step=1), calendar.month_name[5:10], rotation=20)

plt.xlabel("MONTH")
plt.ylabel("RAINFALL")
plt.show()


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export("HW3.ipynb", pdf=False)